In [1]:
# Basic libraries
import numpy as np
import random
import os
from string import punctuation

# Spacy and pickle
import spacy
from pickle import dump, load

# Tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model

# NeuroSholohov
## Takes a part of 'Tihii don' novel and generates text to be like Mihail Sholohov.

# Load 'Tihii Don' novel

In [24]:
with open('Data\TihiiDon.txt', encoding="utf-8") as f:
    td = f.read()
td[:1000]

'\ufeff Михаил Шолохов.\n Тихий Дон\n \n *\xa0КНИГА ПЕРВАЯ *\n \n Не сохами-то славная землюшка наша распахана…\n Распахана наша землюшка лошадиными копытами,\n А засеяна славная землюшка казацкими головами,\n Украшен-то наш тихий Дон молодыми вдовами,\n Цветет наш батюшка тихий Дон сиротами,\n Наполнена волна в тихом Дону отцовскими, материнскими слезами.\n Ой ты, наш батюшка тихий Дон!\n Ой, что же ты, тихий Дон, мутнехонек течешь?\n Ах, как мне, тихому Дону, не мутну течи!\n Со дна меня, тиха Дона, студены ключи бьют,\n Посередь меня, тиха Дона, бела рыбица мутит\n Старинные казачьи песни\n \n *\xa0ЧАСТЬ ПЕРВАЯ * \n \n I\n \n Мелеховский двор — на самом краю хутора. Воротца со скотиньего база ведут на север к Дону. Крутой восьмисаженный спуск меж замшелых в прозелени меловых глыб, и вот берег: перламутровая россыпь ракушек, серая изломистая кайма нацелованной волнами гальки и дальше — перекипающее под ветром вороненой рябью стремя Дона. На восток, за красноталом гуменных плетней,\xa

# Remove titles

In [25]:
titles = ['ЧАСТЬ ПЕРВАЯ', 'ЧАСТЬ ВТОРАЯ',
          'ЧАСТЬ ТРЕТЬЯ', 'ЧАСТЬ ЧЕТВЕРТАЯ',
          'ЧАСТЬ ПЯТАЯ', 'ЧАСТЬ ШЕСТАЯ',
          'ЧАСТЬ СЕДЬМАЯ', 'ЧАСТЬ ВОСЬМАЯ',
          'КНИГА ПЕРВАЯ', 'КНИГА ВТОРАЯ',
          'КНИГА ТРЕТЬЯ', 'КНИГА ЧЕТВЕРТАЯ',
          'Михаил Шолохов', 'I', 'X', 'V'
          ]
for title in titles:
    if td.find(title) >= 0:
        td = td.replace(title, '')
    else:
        print(f'Unable to find: {title}')
td[:200]

'\ufeff .\n Тихий Дон\n \n *\xa0 *\n \n Не сохами-то славная землюшка наша распахана…\n Распахана наша землюшка лошадиными копытами,\n А засеяна славная землюшка казацкими головами,\n Украшен-то наш тихий Дон молодыми'

# Total length of loaded novel

In [26]:
len(td)

2905550

# Use only first 30000 symbols (about 1/10 of the text)

In [27]:
td_short = td[:30000]

In [28]:
nlp = spacy.load('ru_core_news_sm', disable=['parser', 'tagger', 'ner'])

In [29]:
nlp.max_length = 40000

In [30]:
td_tokens = nlp(td_short)
[token.text for token in td_tokens[:50]]

['\ufeff',
 '.',
 '\n ',
 'Тихий',
 'Дон',
 '\n \n ',
 '*',
 '\xa0 ',
 '*',
 '\n \n ',
 'Не',
 'сохами',
 '-',
 'то',
 'славная',
 'землюшка',
 'наша',
 'распахана',
 '…',
 '\n ',
 'Распахана',
 'наша',
 'землюшка',
 'лошадиными',
 'копытами',
 ',',
 '\n ',
 'А',
 'засеяна',
 'славная',
 'землюшка',
 'казацкими',
 'головами',
 ',',
 '\n ',
 'Украшен',
 '-',
 'то',
 'наш',
 'тихий',
 'Дон',
 'молодыми',
 'вдовами',
 ',',
 '\n ',
 'Цветет',
 'наш',
 'батюшка',
 'тихий',
 'Дон']

# Remove punctuation

In [31]:
tokens = [token.text.lower() for token in td_tokens if token.text not in
          punctuation + '\ufeff' + '\n \n \n \n ' + '…' + '\xa0' + '—' + \
          '..' + '  ' + '«' + '»']
tokens[:10]

['тихий',
 'дон',
 'не',
 'сохами',
 'то',
 'славная',
 'землюшка',
 'наша',
 'распахана',
 'распахана']

# Give Neural Network 25 words and predict 26 th

In [32]:
seq_len = 25
train_len = seq_len + 1
dataset = []
for i in range(train_len, len(tokens)):
    seq = tokens[i - train_len:i]
    dataset.append(seq)
dataset[:3]

[['тихий',
  'дон',
  'не',
  'сохами',
  'то',
  'славная',
  'землюшка',
  'наша',
  'распахана',
  'распахана',
  'наша',
  'землюшка',
  'лошадиными',
  'копытами',
  'а',
  'засеяна',
  'славная',
  'землюшка',
  'казацкими',
  'головами',
  'украшен',
  'то',
  'наш',
  'тихий',
  'дон',
  'молодыми'],
 ['дон',
  'не',
  'сохами',
  'то',
  'славная',
  'землюшка',
  'наша',
  'распахана',
  'распахана',
  'наша',
  'землюшка',
  'лошадиными',
  'копытами',
  'а',
  'засеяна',
  'славная',
  'землюшка',
  'казацкими',
  'головами',
  'украшен',
  'то',
  'наш',
  'тихий',
  'дон',
  'молодыми',
  'вдовами'],
 ['не',
  'сохами',
  'то',
  'славная',
  'землюшка',
  'наша',
  'распахана',
  'распахана',
  'наша',
  'землюшка',
  'лошадиными',
  'копытами',
  'а',
  'засеяна',
  'славная',
  'землюшка',
  'казацкими',
  'головами',
  'украшен',
  'то',
  'наш',
  'тихий',
  'дон',
  'молодыми',
  'вдовами',
  'цветет']]

# Create or load tokenizer and save it for the future use

In [33]:
tokenizer_path = 'Models/tokenizer.pcl'

if os.path.exists(tokenizer_path):
    tokenizer = load(open(tokenizer_path, 'rb'))
else:
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(dataset)

In [34]:
dump(tokenizer, open(tokenizer_path, 'wb'))

In [35]:
for i in dataset[0]:
    print(f'{i} : {tokenizer.word_index[i]}')

тихий : 128
дон : 97
не : 7
сохами : 2537
то : 14
славная : 2523
землюшка : 458
наша : 456
распахана : 2527
распахана : 2527
наша : 456
землюшка : 458
лошадиными : 2533
копытами : 457
а : 6
засеяна : 2531
славная : 2523
землюшка : 458
казацкими : 2528
головами : 2526
украшен : 2525
то : 14
наш : 129
тихий : 128
дон : 97
молодыми : 462


# Check tokenized sentences

In [36]:
tokenized = tokenizer.texts_to_sequences(dataset)
tokenized[:1]

[[128,
  97,
  7,
  2537,
  14,
  2523,
  458,
  456,
  2527,
  2527,
  456,
  458,
  2533,
  457,
  6,
  2531,
  2523,
  458,
  2528,
  2526,
  2525,
  14,
  129,
  128,
  97,
  462]]

# Most used words in text

In [37]:
sorted(tokenizer.word_counts.items(), key=lambda x:x[1], reverse=True)[:50]

[('в', 3068),
 ('на', 2986),
 ('и', 2599),
 ('с', 1760),
 ('григорий', 1420),
 ('а', 1419),
 ('не', 1379),
 ('по', 866),
 ('к', 780),
 ('как', 728),
 ('что', 676),
 ('за', 650),
 ('у', 624),
 ('то', 547),
 ('от', 546),
 ('его', 520),
 ('из', 473),
 ('ты', 468),
 ('под', 468),
 ('я', 468),
 ('митька', 442),
 ('он', 416),
 ('ее', 338),
 ('старик', 338),
 ('до', 338),
 ('коня', 326),
 ('прокофий', 312),
 ('она', 312),
 ('пантелей', 312),
 ('вот', 286),
 ('но', 286),
 ('так', 286),
 ('же', 260),
 ('со', 260),
 ('это', 260),
 ('прокофьевич', 260),
 ('ж', 260),
 ('аксинья', 234),
 ('ней', 208),
 ('глаза', 208),
 ('бы', 208),
 ('через', 208),
 ('баркас', 208),
 ('воду', 208),
 ('мне', 182),
 ('жену', 182),
 ('казаки', 182),
 ('голову', 182),
 ('него', 182),
 ('ни', 182)]

# Total number of different words

In [38]:
vocabulary_size = len(tokenizer.word_counts)
vocabulary_size

2538

# Create X and y

In [39]:
sequences = np.array(tokenized)
X = sequences[:, :-1]
y = sequences[:, -1]
X.shape, y.shape

((4353, 25), (4353,))

# Y encoding

In [40]:
y = to_categorical(y, num_classes=vocabulary_size + 1)
y.shape

(4353, 2539)

# Create and train model

In [41]:
def create_model(vocabulary_size, seq_len):
    model = Sequential()
    model.add(Embedding(input_dim=vocabulary_size + 1, output_dim=seq_len,
                        input_length=seq_len))
    model.add(LSTM(units=100, return_sequences=True))
    model.add(LSTM(units=100))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(vocabulary_size + 1, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam',
                  metrics='accuracy')
    return model

model_path = 'Models/my_model_300.h5'
if os.path.exists(model_path):
    model = load_model(model_path)
    model.summary()
else:
    model = create_model(vocabulary_size, seq_len)
    model.summary()
    model.fit(X, y, batch_size=128, epochs=300, verbose=1)
    model.save(model_path)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 25)            63475     
_________________________________________________________________
lstm_2 (LSTM)                (None, 25, 100)           50400     
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 2539)              256439    
Total params: 460,814
Trainable params: 460,814
Non-trainable params: 0
_________________________________________________________________


# Choose random 25 words for NeuroSholohov to continue

In [42]:
random.seed(50)
random_pick = random.randint(0, len(dataset))
seed_text = ' '.join(dataset[random_pick])
seed_text

'шлычка розовая рубаха заправленная в юбку не морщинясь охватывала крутую спину и налитые плечи поднимаясь в гору аксинья клонилась вперед ясно вылегала под рубахой продольная ложбинка'

# Generate new text with NeuroSholohov

In [43]:
output_text = []
for i in range(seq_len):
    # Tokenize the chosen fragment
    encoded_text = tokenizer.texts_to_sequences([seed_text])[0]
    # Remove the first word of fragment
    pad_encoded = pad_sequences([encoded_text], maxlen=seq_len,
                                truncating='pre')
    # Predict the new word
    pred_word_ind = np.argmax(model.predict(pad_encoded, verbose=0)[0], axis=-1)
    # Tokenize it back to the string
    pred_word = tokenizer.index_word[pred_word_ind]
    # Add the new word to the whole string
    seed_text += ' ' + pred_word
    output_text.append(pred_word)

In [44]:
print(' '.join(output_text))

на спине григорий видел бурые круги слинявшей под мышками от пота рубахи провожал глазами каждое движение ему хотелось снова заговорить на ней небось будешь скучать
